In [1]:
import os
os.environ['http_proxy'] = 'http://proxy1.bgc-jena.mpg.de:3128' 
os.environ['https_proxy'] = 'http://proxy1.bgc-jena.mpg.de:3128'

FILES TO UPDATE: 
- config.yaml
- params.yaml
- entity
- configuration manager in src config
- components
- pipeline
- main.py
- app.py

In [2]:
from pathlib import Path
%pwd

'/Net/Groups/BGI/scratch/ppandey/Side_Quest/semantic-land-segment-hf/experiments'

In [3]:
os.chdir('../')
%pwd

'/Net/Groups/BGI/scratch/ppandey/Side_Quest/semantic-land-segment-hf'

In [4]:
from dataclasses import dataclass

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_path: Path
    unzip_dir: Path

In [11]:
from landcover_segmentation.constants import * 
from landcover_segmentation.utils.common import read_yaml, create_directories

In [20]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILEPATH,
        params_filepath = PARAMS_FILEPATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # create artifacts folder
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        # create directory
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_path=config.local_data_path,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config

In [21]:
# COMPONENTS!
import urllib.request as rqst
import zipfile

from landcover_segmentation.logging import logger
from landcover_segmentation.utils.common import get_size

In [28]:
class DataIngestion:
    def __init__(
        self, 
        config: DataIngestionConfig
    ):
        self.config = config
        self.filename = config.local_data_path

    def download_data_from_url(self):
        if not os.path.exists(self.config.local_data_path):
            filename, headers = rqst.urlretrieve(
                url=self.config.source_url,
                filename=self.filename
            )
            logger.info(f'{filename} downloaded! Information: \n{headers}')

        else:
            logger.info(f'{self.filename} already exists!')

    def extract_zip_file(self):
        unzip_data_path = self.config.unzip_dir
        os.makedirs(unzip_data_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_path, 'r') as zip_ref:
            zip_ref.extractall(unzip_data_path)       

In [29]:
# Ingestion pipeline
try:
    config = ConfigurationManager()
    data_ing_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ing_config)
    data_ingestion.download_data_from_url()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-05-12 10:45:05,780: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-05-12 10:45:05,784: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-12 10:45:05,785: INFO: common: created directory at: artifacts]
[2024-05-12 10:45:05,785: INFO: common: created directory at: artifacts/data_ingestion]
[2024-05-12 10:45:05,785: INFO: 1184886628: artifacts/data_ingestion/landcover_data.zip already exists!]
